In [1]:
import pandas as pd
import muon as mu
import scanpy as sc
from muon import atac as ac
import numpy as np
import warnings
import pychromvar as pc
import sys
import os
import pickle

/root/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
file_dir = os.path.abspath('')
data_path = os.path.join(file_dir, '..', '..', 're_design', '10x_data')

h5_file_path = os.path.join(data_path, 'pbmc3k_multi.h5mu')
save_path = os.path.join(file_dir, 'generated_data')

In [44]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    mdata = mu.read_h5mu(h5_file_path)
mdata

MuData object with n_obs × n_vars = 3217 × 32907
  obs:	'leiden_wnn'
  uns:	'leiden', 'leiden_wnn_colors', 'umap', 'wnn'
  obsm:	'X_umap', 'X_wnn_umap'
  obsp:	'wnn_connectivities', 'wnn_distances'
  3 modalities
    gene:	3217 x 15863
      obs:	'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden_umap', 'leiden_euclidean', 'leiden_wnn', 'tf_leiden_umap', 'tf_leiden_euclidean'
      var:	'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std', 'gene_ids_simple', 'tf'
      uns:	'hvg', 'leiden', 'leiden_umap_colors', 'leiden_umap_sizes', 'log1p', 'neighbors_euclidean', 'neighbors_umap', 'paga', 'pca', 'rank_loci_groups', 'tf_neighbors_euclidean', 'tf_neighbors_umap', 'umap'
      obsm:	'X_pca', 'X_umap', 'tf_pca'
      varm:	'PCs'
      layers:	'counts'
      obsp:	'neighbors_euclidean_connectivities', 'neighbors_euclidean_distances', 'neighbors_umap_connectivities', 'neighbors_umap_distances', 'tf_neighbors_umap_connectivities', 'tf_neighbors_umap_distances'
    cre:	3217 x 16298
      obs:	'n_genes_by_counts', 'total_counts', 'avg_frag_length', 'leiden_umap', 'leiden_umap20', 'leiden_umap50', 'leiden_euclidean', 'leiden_wnn'
      var:	'directionality', 'orientation', 'bias_strand', 'fwd_count', 'rev_count', 'chrom', 'summit_center', 'abs_summit_diff', 'oritn_summit_diff', 'start', 'end', 'strand', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'frag_length', 'gc_bias'
      uns:	'hvg', 'leiden', 'leiden_euclidean_colors', 'leiden_umap_colors', 'leiden_umap_sizes', 'log1p', 'motif_name', 'neighbors_euclidean', 'neighbors_umap', 'paga', 'pca', 'peak_seq', 'rank_loci_groups', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs', 'bg_peaks', 'motif_match'
      layers:	'counts'
      obsp:	'neighbors_euclidean_connectivities', 'neighbors_euclidean_distances', 'neighbors_umap_connectivities', 'neighbors_umap_distances'
    chromvar:	3217 x 746
      obs:	'leiden_wnn'
      uns:	'rank_genes_groups'

# Kmer clustering

In [45]:
def cut_sequences_midpoint(data, seqlen):
    half = round(seqlen/2)
    for idx, row in data.iterrows():
        ms = row['summit_center'] - row['start'] # mid - start
        em = row['end'] - row['summit_center'] # end - mid
        seq = row['sequence']
        if ms < half:
            data.loc[idx, 'sequence'] = seq[:seqlen]
        elif em < half:
            data.loc[idx, 'sequence'] = seq[-seqlen:]
        else:
            data.loc[idx, 'sequence'] = seq[ms-half:ms+half]
    return data

In [46]:
seq_len = 200

In [47]:
mdata['cre'].var['sequence'] = mdata.mod['cre'].uns['peak_seq']
cre_var = mdata['cre'].var.reset_index(names='peak')
# subset only required columns
cre_var = cre_var[['peak','chrom','start','end','summit_center','sequence']]
# cut sequences for clustering
cre_var = cut_sequences_midpoint(cre_var, seq_len)
cre_var

,peak,chrom,start,end,summit_center,sequence
0,chr1_29236_29737_-,chr1,29236,29737,29337,TCCCTCCAGCCCCTCCGGGTCCCCTACTTCGCCCCGCCAGGCCCCC...
1,chr1_199774_200275_-,chr1,199774,200275,200006,CCGCCCACAGTCCGCCCGCGCCTCCGGGTCCTAACGCCGCCGCTCG...
2,chr1_629102_629950_+,chr1,629102,629950,629793,ATCACAGCGCTAAGCTCGCACTGATTTTTTACCTGAGTAGGCCTAG...
3,chr1_630597_631171_+,chr1,630597,631171,630948,CACAAACACTTAGTTAACAGCTAAGCACCCTAATCAACTGGCTTCA...
4,chr1_632356_633841_+,chr1,632356,633841,633051,ACTATCCTGCCCGCCATCATCCTAGTCCTTATCGCCCTCCCATCCC...
...,...,...,...,...,...,...
16293,chrY_21028687_21029188_-,chrY,21028687,21029188,21028788,GTGGTGAGTGTTATAGCTCATAATGTTGGCACGGACCCAAACAGTT...
16294,chrY_21138232_21138733_+,chrY,21138232,21138733,21138531,AGGGCTGAGGATGGCTTATCAAAAACAACCCACTTTATACAAGGAA...
16295,chrY_21254335_21254836_+,chrY,21254335,21254836,21254594,TAACACCTGCCTTCTATGAGTTAGGAATAATTTTCTCTTCCTCAAT...
16296,chrY_57067464_57067965_+,chrY,57067464,57067965,57067865,GAACGCAGCAGGCCTAGCCGTGTCGCCTGCTGCCATTGGAGGAGCG...


In [48]:
sequences = np.array(cre_var['sequence'])
len(sequences)

16298

In [49]:
from collections import Counter 

counter = Counter()

for seq in sequences:
    counter.update(seq)

In [50]:
counter

Counter({'G': 1056507, 'C': 1049517, 'T': 577342, 'A': 576234})

In [51]:
idx_with_n = np.where(pd.Series(sequences).str.contains('N'))
sequences = np.delete(sequences, idx_with_n)
cre_var = cre_var.drop(cre_var.index[idx_with_n])
len(sequences)

16298

In [52]:
n_components = 50
k = 5

In [53]:
from sklearn.feature_extraction.text import CountVectorizer


# Create all possible k-mers from 'A', 'C', 'G', 'T'
vectorizer = CountVectorizer(analyzer='char', ngram_range=(k, k))

# Fit the vectorizer on the combined set
kmer_embed = vectorizer.fit_transform(sequences)

In [54]:
kmer_embed

<16298x1024 sparse matrix of type '<class 'numpy.int64'>'
	with 2538938 stored elements in Compressed Sparse Row format>

In [55]:
# normalize
kmer_embed = kmer_embed/ (seq_len - k + 1)

In [56]:
from sklearn.decomposition import PCA


# Apply PCA transformation
pca = PCA(n_components=n_components, svd_solver = "arpack")
# Fit PCA on the combined data to ensure the same transformation is applied to both sets
pca_embed = pca.fit_transform(kmer_embed)

print("Fraction of total variance explained by PCA components: ", np.sum(pca.explained_variance_ratio_))

Fraction of total variance explained by PCA components:  0.4403534710672228


In [57]:
pca_embed.shape

(16298, 50)

In [58]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# scaler = StandardScaler()
# embed_scaled = scaler.fit_transform(pca_embed)

Z = linkage(pca_embed, method='ward', metric='euclidean')  # method 'average', 'complete'

In [59]:
num_clusters = 3
clusters = fcluster(Z, num_clusters, criterion='maxclust')

# Create a dictionary of loci to clusters
cre_cluster = dict(zip(mdata.mod['cre'].var.index, clusters))

pd.DataFrame.from_dict(cre_cluster, orient='index')[0].value_counts()

0
3    8077
1    6520
2    1701
Name: count, dtype: int64

# Final data

In [66]:
full_data = cre_var.copy()

# new column cluster by mapping genes to cluster with help of gene_cluster dict
full_data['cell_type'] = full_data['peak'].map(cre_cluster)
full_data = full_data[['peak','chrom','sequence','cell_type']]
#full_data = full_data.explode('cell_type')
full_data = full_data[full_data['cell_type'].notna()]

full_data['cell_type'] = "ct"+(full_data.cell_type.astype(int)).astype(str)
full_data.reset_index(drop=True,inplace=True)
# for data constitencty with legacy code
full_data

,peak,chrom,sequence,cell_type
0,chr1_29236_29737_-,chr1,TCCCTCCAGCCCCTCCGGGTCCCCTACTTCGCCCCGCCAGGCCCCC...,ct3
1,chr1_199774_200275_-,chr1,CCGCCCACAGTCCGCCCGCGCCTCCGGGTCCTAACGCCGCCGCTCG...,ct3
2,chr1_629102_629950_+,chr1,ATCACAGCGCTAAGCTCGCACTGATTTTTTACCTGAGTAGGCCTAG...,ct1
3,chr1_630597_631171_+,chr1,CACAAACACTTAGTTAACAGCTAAGCACCCTAATCAACTGGCTTCA...,ct1
4,chr1_632356_633841_+,chr1,ACTATCCTGCCCGCCATCATCCTAGTCCTTATCGCCCTCCCATCCC...,ct1
...,...,...,...,...
16293,chrY_21028687_21029188_-,chrY,GTGGTGAGTGTTATAGCTCATAATGTTGGCACGGACCCAAACAGTT...,ct1
16294,chrY_21138232_21138733_+,chrY,AGGGCTGAGGATGGCTTATCAAAAACAACCCACTTTATACAAGGAA...,ct1
16295,chrY_21254335_21254836_+,chrY,TAACACCTGCCTTCTATGAGTTAGGAATAATTTTCTCTTCCTCAAT...,ct1
16296,chrY_57067464_57067965_+,chrY,GAACGCAGCAGGCCTAGCCGTGTCGCCTGCTGCCATTGGAGGAGCG...,ct1


In [67]:
full_data['sequence'].str.len().describe()

count    16298.0
mean       200.0
std          0.0
min        200.0
25%        200.0
50%        200.0
75%        200.0
max        200.0
Name: sequence, dtype: float64

In [68]:
full_data.isna().any()

peak         False
chrom        False
sequence     False
cell_type    False
dtype: bool

In [69]:
print("Total peaks:", full_data.shape[0],"; more than 1 cluster:" ,full_data[full_data['peak'].duplicated()].shape[0])

Total peaks: 16298 ; more than 1 cluster: 0


In [70]:
print("File statistics")
print(full_data['cell_type'].value_counts())
print(full_data['chrom'].value_counts())

File statistics
cell_type
ct3    8077
ct1    6520
ct2    1701
Name: count, dtype: int64
chrom
chr1     1617
chr19    1229
chr2     1129
chr17    1042
chr11     889
chr3      866
chr6      863
chr7      854
chr12     843
chr16     733
chr5      723
chr14     615
chr9      612
chr10     597
chr4      571
chr8      531
chr15     506
chrX      480
chr20     425
chr22     397
chr13     284
chr18     225
chr21     208
chrY       39
chrM       20
Name: count, dtype: int64


In [71]:
full_data[full_data['chrom']=='chr1']['cell_type'].value_counts()

cell_type
ct3    804
ct1    674
ct2    139
Name: count, dtype: int64

In [72]:
file_path = os.path.join(save_path, 'tcre_seq_kmer_cluster.csv')

full_data.to_csv(file_path, index=False)